# COX REGRESSION (sklearn adapter)

In [22]:
# BEST data
import pandas as pd

vgg_df = pd.read_excel("vgg_final.xlsx", engine='openpyxl')
target_df = pd.read_excel("vgg_labels.xlsx", engine='openpyxl')

common_indices = vgg_df['PP']

## EHR

In [23]:
directory = r"L:\basic\divi\jstoker\slicer_pdac\Master Students SS 23\Mattia\survival.xlsx"
ehr = pd.read_excel(directory, usecols='B,C,H,I,J,K,L', nrows=137).iloc[1:,:]

#change to int
ehr['Randomisatie nummer'] = ehr['Randomisatie nummer'].astype('int')

#create new_column_id
ehr['PP'] = ehr['Studie'] + '-' + ehr['Randomisatie nummer'].astype(str)

#drop unecessary columnns
ehr = ehr.drop(['Randomisatie nummer', 'Studie'], axis=1)

ehr = ehr.set_index('PP')

ehr = ehr.loc[common_indices]

new_columns = {ehr.columns[0]: 'age', ehr.columns[1]: 'tumor diameter', ehr.columns[2]: 'tumor location', ehr.columns[3]: 'resection margin', ehr.columns[4]: 'nat'}
ehr = ehr.rename(columns = new_columns)

ehr

,age,tumor diameter,tumor location,resection margin,nat
PP,,,,,
PP1-106,67.232877,3.3,3,none,1
PP1-125,63.468493,3.5,1,none,3
PP1-136,68.898630,3.5,1,R0,1
PP1-137,59.961644,2.7,1,R1,3
PP1-145,68.693151,4.0,4,R0,3
...,...,...,...,...,...
PP2-77,63.106849,3.5,4,R1,2
PP2-8,66.153425,NaN,1,R0,3
PP2-87,54.260274,4.7,2,R0,3


#### tumor location

In [24]:
#change 2.6 with mode
mode = int(ehr['tumor location'].mode().values)
ehr.loc['PP2-321', 'tumor location'] = mode
ehr.loc['PP2-321']

age                 60.380822
tumor diameter            1.0
tumor location              1
resection margin           R0
nat                         2
Name: PP2-321, dtype: object

In [25]:
ehr['tumor location'].unique()

array([3, 1, 4, '1 *Groove', 1.3, 2, 1.2], dtype=object)

In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Replacing '1 *Groove' with 1
ehr['tumor location'] = ehr['tumor location'].replace('1 *Groove', 1)

# Creating two new columns '2' and '3' for categories 2 and 3
ehr['2'] = 0
ehr['3'] = 0

# Update these columns whenever we encounter 1.2 or 1.3
ehr.loc[ehr['tumor location'] == 1.2, '2'] = 1
ehr.loc[ehr['tumor location'] == 1.3, '3'] = 1

# Also replacing 1.2 and 1.3 with 1
ehr['tumor location'] = ehr['tumor location'].replace([1.2, 1.3], 1)

# One-hot-encode the 'tumor location'
onehotencoder = OneHotEncoder()
ehr_encoded = onehotencoder.fit_transform(ehr['tumor location'].values.reshape(-1,1)).toarray()

# Creating a DataFrame from our encoded array
ehr_encoded = pd.DataFrame(ehr_encoded, columns = ["tumor_location_"+str(int(i)) for i in range(ehr_encoded.shape[1])])

# set same index also for ehr_encoded
ehr_encoded = ehr_encoded.set_index(ehr.index)

# Concatenating the original DataFrame and the one-hot-encoded DataFrame
ehr = pd.concat([ehr, ehr_encoded], axis=1)

#adding values from column 2 and 3
ehr['tumor_location_2'] = ehr['tumor_location_2'] + ehr['2']
ehr['tumor_location_3'] = ehr['tumor_location_3'] + ehr['3']

# Dropping the '2' and '3' columns
ehr = ehr.drop(columns=['2', '3', 'tumor location'])

#### resection margin

In [27]:
ehr['resection margin'].unique()

array(['none', 'R0', 'R1'], dtype=object)

In [28]:
onehotencoder = OneHotEncoder()
ehr_encoded = onehotencoder.fit_transform(ehr['resection margin'].values.reshape(-1,1)).toarray()

# Creating a DataFrame from our encoded array
ehr_encoded = pd.DataFrame(ehr_encoded, columns = ["resection_margin_"+str(int(i)) for i in range(ehr_encoded.shape[1])])

# set same index also for ehr_encoded
ehr_encoded = ehr_encoded.set_index(ehr.index)

# Concatenating the original DataFrame and the one-hot-encoded DataFrame
ehr = pd.concat([ehr, ehr_encoded], axis=1)

# drop
ehr = ehr.drop(columns=['resection margin'])

#### nat

In [29]:
ehr['nat'].unique()

array([1, 3, 2, nan], dtype=object)

In [30]:
# Calculate the mode of the 'nat' column
mode_value = ehr['nat'].mode()[0]

# Fill NaN values with the mode value
ehr['nat'] = ehr['nat'].fillna(mode_value)

onehotencoder = OneHotEncoder()
ehr_encoded = onehotencoder.fit_transform(ehr['nat'].values.reshape(-1,1)).toarray()

# Creating a DataFrame from our encoded array
ehr_encoded = pd.DataFrame(ehr_encoded, columns = ["nat_"+str(int(i)) for i in range(ehr_encoded.shape[1])])

# set same index also for ehr_encoded
ehr_encoded = ehr_encoded.set_index(ehr.index)

# Concatenating the original DataFrame and the one-hot-encoded DataFrame
ehr = pd.concat([ehr, ehr_encoded], axis=1)

# drop
ehr = ehr.drop(columns=['nat'])

In [31]:
ehr.head()

,age,tumor diameter,tumor_location_0,tumor_location_1,tumor_location_2,tumor_location_3,resection_margin_0,resection_margin_1,resection_margin_2,nat_0,nat_1,nat_2
PP,,,,,,,,,,,,
PP1-106,67.232877,3.3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
PP1-125,63.468493,3.5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
PP1-136,68.898630,3.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
PP1-137,59.961644,2.7,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
PP1-145,68.693151,4.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [32]:
#WEIIIIIRDDDDDD
ehr[ehr['nat_0']==1]

,age,tumor diameter,tumor_location_0,tumor_location_1,tumor_location_2,tumor_location_3,resection_margin_0,resection_margin_1,resection_margin_2,nat_0,nat_1,nat_2
PP,,,,,,,,,,,,
PP1-106,67.232877,3.3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
PP1-136,68.898630,3.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
PP1-150,55.838356,2.8,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
PP1-16,67.109589,3.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
PP1-2,54.591781,2.5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
PP1-74,64.487671,3.7,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
PP1-96,56.224658,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
PP2-164,76.076712,4.2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
PP2-21,73.887671,2.9,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [33]:
ehr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102 entries, PP1-106 to PP2-98
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   age                 102 non-null    float64
 1   tumor diameter      101 non-null    float64
 2   tumor_location_0    102 non-null    float64
 3   tumor_location_1    102 non-null    float64
 4   tumor_location_2    102 non-null    float64
 5   tumor_location_3    102 non-null    float64
 6   resection_margin_0  102 non-null    float64
 7   resection_margin_1  102 non-null    float64
 8   resection_margin_2  102 non-null    float64
 9   nat_0               102 non-null    float64
 10  nat_1               102 non-null    float64
 11  nat_2               102 non-null    float64
dtypes: float64(12)
memory usage: 14.4+ KB


#### tumor diamater

In [34]:
# Calculate the mean of the 'nat' column
mean_value = ehr['tumor diameter'].mean()

# Fill NaN values with the mode value
ehr['tumor diameter'] = ehr['tumor diameter'].fillna(mean_value)

### Concatenate with dataset

In [35]:
merged_df = pd.concat([vgg_df.set_index('PP'), ehr, target_df.set_index('PP')], axis=1)

In [36]:
merged_df.head()

,0,1,2,3,4,5,6,7,8,9,...,tumor_location_2,tumor_location_3,resection_margin_0,resection_margin_1,resection_margin_2,nat_0,nat_1,nat_2,Event,Duration
PP,,,,,,,,,,,,,,,,,,,,,
PP1-106,0.0,0.0,0,0.228759,0.349095,0.668154,1.428231,0.000000,0.0,0.308558,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1,428
PP1-125,0.0,0.0,0,0.000000,0.000000,0.000000,1.436633,0.000000,0.0,6.106245,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1,91
PP1-136,0.0,0.0,0,1.517700,0.000000,0.000000,2.948053,3.862043,0.0,0.000000,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,58
PP1-137,0.0,0.0,0,0.000000,0.000000,1.417175,3.569140,0.000000,0.0,3.927991,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,1479
PP1-145,0.0,0.0,0,0.000000,2.280414,0.595382,0.000000,0.120114,0.0,4.016916,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1,1627


In [37]:
merged_df.iloc[:,4098:]

,tumor_location_0,tumor_location_1,tumor_location_2,tumor_location_3,resection_margin_0,resection_margin_1,resection_margin_2,nat_0,nat_1,nat_2,Event,Duration
PP,,,,,,,,,,,,
PP1-106,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1,428
PP1-125,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1,91
PP1-136,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,58
PP1-137,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,1479
PP1-145,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1,1627
...,...,...,...,...,...,...,...,...,...,...,...,...
PP2-77,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1,414
PP2-8,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1,408
PP2-87,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1,368


## Modeling with Train and Test splits

#### repeated cross validation

In [38]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from lifelines.utils.sklearn_adapter import sklearn_adapter
from lifelines import CoxPHFitter
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold

X = merged_df.drop('Duration', axis=1)  # keep as a dataframe
y = merged_df['Duration']

# Define the number of folds and repeats, and an empty list to store the scores
num_folds = 5
num_repeats = 10
scores = []

for r in range(num_repeats):
    print(f"--- Repeat: {r + 1} ---")
    
    # Generate indices for splits
    np.random.seed(r+420)  # change the seed for each repeat
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    fold_sizes = (len(X) // num_folds) * np.ones(num_folds, dtype=int)  # equally divide indices
    fold_sizes[:len(X) % num_folds] += 1  # if len(X) is not exactly divisible by num_folds, assign remainder to first few

    current = 0
    splits = []
    for fold_size in fold_sizes:
        start, stop = current, current + fold_size
        splits.append((indices[start:stop], np.concatenate((indices[:start], indices[stop:]))))  # (test, train)
        current = stop

    # Perform cross-validation
    for i, (test_idx, train_idx) in enumerate(splits):
        print(f"Fold: {i + 1}")
        
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Drop 'Event' column
        X_train_new = X_train.drop('Event', axis=1)
        X_test_new = X_test.drop('Event', axis=1)
        
        #change to array
        X_train_new = X_train_new.values
        X_test_new = X_test_new.values
        
        # Scale the data
        scaler = StandardScaler()
        scaler = MinMaxScaler()
        X_train_new[:, :4098] = scaler.fit_transform(X_train_new[:, :4098])
        X_test_new[:, :4098] = scaler.transform(X_test_new[:, :4098])
        
        # Perform PCA
        pca = PCA(n_components=25)
        X_train_new_pca = pca.fit_transform(X_train_new[:, :4096])
        X_test_new_pca = pca.transform(X_test_new[:, :4096])
        X_train_new = np.concatenate((X_train_new_pca, X_train_new[:,4096:]), axis=1)
        X_test_new = np.concatenate((X_test_new_pca, X_test_new[:,4096:]), axis=1)
        
        # Convert back to DataFrame
        X_train_new = pd.DataFrame(X_train_new)
        X_test_new = pd.DataFrame(X_test_new)
        
        # Add back indices
        X_train_new.index = X_train.index
        X_test_new.index = X_test.index
        
        # Add 'Event' column back
        X_train_new['Event'] = X_train['Event']
        X_test_new['Event'] = X_test['Event']
        
        # Instantiate CoxRegression object
        CoxRegression = sklearn_adapter(CoxPHFitter, event_col='Event')
        sk_cph = CoxRegression(penalizer=1e-5)
        sk_cph.fit(X_train_new, y_train)
        
        # Calculate and store the score
        score = sk_cph.score(X_test_new, y_test)
        scores.append(score)

        print(f"Score: {score}")
        print("-----------------------------")

# Calculate mean
mean_score = np.mean(scores)

print(f"Mean Score: {mean_score}")

--- Repeat: 1 ---
Fold: 1
Score: 0.6296296296296297
-----------------------------
Fold: 2
Score: 0.581151832460733
-----------------------------
Fold: 3
Score: 0.74375
-----------------------------
Fold: 4
Score: 0.5988372093023255
-----------------------------
Fold: 5


C:\Users\mcintioli\AppData\Local\anaconda3\envs\thesis\lib\site-packages\lifelines\utils\__init__.py:1122: ConvergenceWarning: Column 33 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['Event'].astype(bool)
>>> print(df.loc[events, '33'].var())
>>> print(df.loc[~events, '33'].var())

A very low variance means that the column 33 completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\mcintioli\AppData\Local\anaconda3\envs\thesis\lib\site-packages\lifelines\fitters\coxph_fitter.py:1611: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples

Score: 0.8314606741573034
-----------------------------
--- Repeat: 2 ---
Fold: 1


C:\Users\mcintioli\AppData\Local\anaconda3\envs\thesis\lib\site-packages\lifelines\utils\__init__.py:1122: ConvergenceWarning: Column 33 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['Event'].astype(bool)
>>> print(df.loc[events, '33'].var())
>>> print(df.loc[~events, '33'].var())

A very low variance means that the column 33 completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\mcintioli\AppData\Local\anaconda3\envs\thesis\lib\site-packages\lifelines\fitters\coxph_fitter.py:1611: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples

Score: 0.7090909090909091
-----------------------------
Fold: 2


...

C:\Users\mcintioli\AppData\Local\anaconda3\envs\thesis\lib\site-packages\lifelines\utils\__init__.py:1122: ConvergenceWarning: Column 33 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['Event'].astype(bool)
>>> print(df.loc[events, '33'].var())
>>> print(df.loc[~events, '33'].var())

A very low variance means that the column 33 completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\mcintioli\AppData\Local\anaconda3\envs\thesis\lib\site-packages\lifelines\fitters\coxph_fitter.py:1611: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples

Score: 0.7251461988304093
-----------------------------
Fold: 4
Score: 0.5405405405405406
-----------------------------
Fold: 5
Score: 0.6777777777777778
-----------------------------
--- Repeat: 10 ---
Fold: 1
Score: 0.75
-----------------------------
Fold: 2
Score: 0.7287234042553191
-----------------------------
Fold: 3
Score: 0.5384615384615384
-----------------------------
Fold: 4


C:\Users\mcintioli\AppData\Local\anaconda3\envs\thesis\lib\site-packages\lifelines\utils\__init__.py:1122: ConvergenceWarning: Column 33 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['Event'].astype(bool)
>>> print(df.loc[events, '33'].var())
>>> print(df.loc[~events, '33'].var())

A very low variance means that the column 33 completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\mcintioli\AppData\Local\anaconda3\envs\thesis\lib\site-packages\lifelines\fitters\coxph_fitter.py:1611: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples

Score: 0.7570621468926554
-----------------------------
Fold: 5
Score: 0.7628205128205128
-----------------------------
Mean Score: 0.6727068445803047


C:\Users\mcintioli\AppData\Local\anaconda3\envs\thesis\lib\site-packages\lifelines\utils\__init__.py:1122: ConvergenceWarning: Column 33 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['Event'].astype(bool)
>>> print(df.loc[events, '33'].var())
>>> print(df.loc[~events, '33'].var())

A very low variance means that the column 33 completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\mcintioli\AppData\Local\anaconda3\envs\thesis\lib\site-packages\lifelines\fitters\coxph_fitter.py:1611: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples

# Random Forest Survival

#### 10 different cv with 5 folds

In [44]:
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sksurv.ensemble import RandomSurvivalForest
from sklearn.decomposition import PCA

X = merged_df.drop(['Duration', 'Event'], axis=1)  # keep as a dataframe
y = merged_df[['Event','Duration']]

#change y to array of tuples (Event, Duration)
y = np.array([(bool(arr[0]), arr[1]) for arr in y.values], dtype=[('boolean', bool), ('integer', int)])

# Define the number of folds, repeats and an empty list to store the scores
num_folds = 5
num_repeats = 10
scores = []

# Create the RepeatedKFold object
rkf = RepeatedKFold(n_splits=num_folds, n_repeats=num_repeats, random_state=1)

# Perform Repeated K-Fold cross-validation
for i, (train_index, test_index) in enumerate(rkf.split(X)):
    print(f"Fold: {(i % num_folds) + 1}, Repeat: {i // num_folds + 1}")
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
     #change to array
    X_train_new = X_train.values
    X_test_new = X_test.values
    
    # Scale the data
    scaler = StandardScaler()
    X_train_new[:, :4098] = scaler.fit_transform(X_train_new[:, :4098])
    X_test_new[:, :4098] = scaler.transform(X_test_new[:, :4098])
    
    # Perform PCA
    pca = PCA(n_components=24)
    X_train_new_pca = pca.fit_transform(X_train_new[:, :4096])
    X_test_new_pca = pca.transform(X_test_new[:, :4096])
    X_train_new = np.concatenate((X_train_new_pca, X_train_new[:,4096:]), axis=1)
    X_test_new = np.concatenate((X_test_new_pca, X_test_new[:,4096:]), axis=1)
    
    # Convert back to DataFrame
    X_train_new = pd.DataFrame(X_train_new)
    X_test_new = pd.DataFrame(X_test_new)
    
    # Add back indices
    X_train_new.index = X_train.index
    X_test_new.index = X_test.index
    
    # Instantiate RandomSurvivalForest object
    rsf = RandomSurvivalForest()
    rsf.fit(X_train_new, y_train)
    
    # Calculate and store the score
    score = rsf.score(X_test_new, y_test)
    scores.append(score)
    
    print(f"Score: {score}")
    print("-----------------------------")

# Calculate mean and 95% confidence interval of the scores
mean_score = np.mean(scores)

print(f"Mean Score: {mean_score}")

Fold: 1, Repeat: 1
Score: 0.7664233576642335
-----------------------------
Fold: 2, Repeat: 1
Score: 0.7865853658536586
-----------------------------
Fold: 3, Repeat: 1
Score: 0.659217877094972
-----------------------------
Fold: 4, Repeat: 1
Score: 0.5423728813559322
-----------------------------
Fold: 5, Repeat: 1
Score: 0.7473118279569892
-----------------------------
Fold: 1, Repeat: 2
Score: 0.725
-----------------------------
Fold: 2, Repeat: 2
Score: 0.6280487804878049
-----------------------------
Fold: 3, Repeat: 2
Score: 0.6611111111111111
-----------------------------
Fold: 4, Repeat: 2
Score: 0.710691823899371
-----------------------------
Fold: 5, Repeat: 2
Score: 0.695364238410596
-----------------------------
Fold: 1, Repeat: 3
Score: 0.7894736842105263
-----------------------------
Fold: 2, Repeat: 3
Score: 0.7538461538461538
-----------------------------
Fold: 3, Repeat: 3
Score: 0.5879120879120879
-----------------------------
Fold: 4, Repeat: 3
Score: 0.7987012987012

# GDB survival (ComponentwiseGradientBoostingSurvivalAnalysis)

#### 10 different cv with 5 folds

In [45]:
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sksurv.ensemble import ComponentwiseGradientBoostingSurvivalAnalysis
from sklearn.decomposition import PCA

X = merged_df.drop(['Duration', 'Event'], axis=1)  # keep as a dataframe
y = merged_df[['Event','Duration']]

#change y to array of tuples (Event, Duration)
y = np.array([(bool(arr[0]), arr[1]) for arr in y.values], dtype=[('boolean', bool), ('integer', int)])

# Define the number of folds, repeats and an empty list to store the scores
num_folds = 5
num_repeats = 10
scores = []

# Create the RepeatedKFold object
rkf = RepeatedKFold(n_splits=num_folds, n_repeats=num_repeats, random_state=1)

# Perform Repeated K-Fold cross-validation
for i, (train_index, test_index) in enumerate(rkf.split(X)):
    print(f"Fold: {(i % num_folds) + 1}, Repeat: {i // num_folds + 1}")
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    #change to array
    X_train_new = X_train.values
    X_test_new = X_test.values
    
    # Scale the data
    scaler = StandardScaler()
    scaler = MinMaxScaler()
    X_train_new[:, :4098] = scaler.fit_transform(X_train_new[:, :4098])
    X_test_new[:, :4098] = scaler.transform(X_test_new[:, :4098])
    
    # Perform PCA
    pca = PCA(n_components=25)
    X_train_new_pca = pca.fit_transform(X_train_new[:, :4096])
    X_test_new_pca = pca.transform(X_test_new[:, :4096])
    X_train_new = np.concatenate((X_train_new_pca, X_train_new[:,4096:]), axis=1)
    X_test_new = np.concatenate((X_test_new_pca, X_test_new[:,4096:]), axis=1)
    
    # Convert back to DataFrame
    X_train_new = pd.DataFrame(X_train_new)
    X_test_new = pd.DataFrame(X_test_new)
    
    # Add back indices
    X_train_new.index = X_train.index
    X_test_new.index = X_test.index
    
    # Instantiate GB object
    gdb = ComponentwiseGradientBoostingSurvivalAnalysis(loss="coxph")
    gdb.fit(X_train_new, y_train)
    
    # Calculate and store the score
    score = gdb.score(X_test_new, y_test)
    scores.append(score)
    
    print(f"Score: {score}")
    print("-----------------------------")

# Calculate mean and 95% confidence interval of the scores
mean_score = np.mean(scores)

print(f"Mean Score: {mean_score}")

Fold: 1, Repeat: 1
Score: 0.8102189781021898
-----------------------------
Fold: 2, Repeat: 1
Score: 0.5914634146341463
-----------------------------
Fold: 3, Repeat: 1
Score: 0.6256983240223464
-----------------------------
Fold: 4, Repeat: 1
Score: 0.5536723163841808
-----------------------------
Fold: 5, Repeat: 1
Score: 0.7096774193548387
-----------------------------
Fold: 1, Repeat: 2
Score: 0.74
-----------------------------
Fold: 2, Repeat: 2
Score: 0.573170731707317
-----------------------------
Fold: 3, Repeat: 2
Score: 0.6277777777777778
-----------------------------
Fold: 4, Repeat: 2
Score: 0.660377358490566
-----------------------------
Fold: 5, Repeat: 2
Score: 0.6754966887417219
-----------------------------
Fold: 1, Repeat: 3
Score: 0.7543859649122807
-----------------------------
Fold: 2, Repeat: 3
Score: 0.6820512820512821
-----------------------------
Fold: 3, Repeat: 3
Score: 0.521978021978022
-----------------------------
Fold: 4, Repeat: 3
Score: 0.81818181818181